### Time Series Analysis on Facebook Prophet Model for VM 1



In [2]:
import pandas as pd
import datetime
import seaborn as sns

In [3]:
from fbprophet import Prophet

In [4]:
from pandas import DataFrame
from pandas import to_datetime

In [5]:
df=  pd.read_csv('/content/ProcessedData.csv')
df.head()

,Time,Timestamp [ms],CPU cores,CPU capacity provisioned [MHZ],CPU usage [MHZ],CPU usage [%],Memory capacity provisioned [KB],Memory usage [KB],Disk read throughput [KB/s],Disk write throughput [KB/s],Network received throughput [KB/s],Network transmitted throughput [KB/s]
0,2013-06-30 22:03:24,1372629804,2,5851.9989,87.779984,1.500000,8218624.0,1.034593e+06,160.866667,21.733333,0.266667,1.466667
1,2013-06-30 22:08:24,1372630104,2,5851.9989,29.259995,0.500000,8218624.0,4.585755e+05,0.000000,2.333333,0.200000,1.000000
2,2013-06-30 22:13:24,1372630404,2,5851.9989,27.309328,0.466667,8218624.0,1.845480e+05,32.066667,4.200000,0.133333,1.066667
3,2013-06-30 22:18:24,1372630704,2,5851.9989,23.407996,0.400000,8218624.0,7.829227e+04,0.000000,0.866667,0.066667,1.000000
4,2013-06-30 22:23:24,1372631004,2,5851.9989,19.506663,0.333333,8218624.0,1.677720e+05,0.000000,0.200000,0.133333,1.000000


Calculating Memory Usage %


In [6]:
df['Memory usage [%]']=(df['Memory usage [KB]'].div(df['Memory capacity provisioned [KB]']))*100
df.head()

,Time,Timestamp [ms],CPU cores,CPU capacity provisioned [MHZ],CPU usage [MHZ],CPU usage [%],Memory capacity provisioned [KB],Memory usage [KB],Disk read throughput [KB/s],Disk write throughput [KB/s],Network received throughput [KB/s],Network transmitted throughput [KB/s],Memory usage [%]
0,2013-06-30 22:03:24,1372629804,2,5851.9989,87.779984,1.500000,8218624.0,1.034593e+06,160.866667,21.733333,0.266667,1.466667,12.588394
1,2013-06-30 22:08:24,1372630104,2,5851.9989,29.259995,0.500000,8218624.0,4.585755e+05,0.000000,2.333333,0.200000,1.000000,5.579711
2,2013-06-30 22:13:24,1372630404,2,5851.9989,27.309328,0.466667,8218624.0,1.845480e+05,32.066667,4.200000,0.133333,1.066667,2.245485
3,2013-06-30 22:18:24,1372630704,2,5851.9989,23.407996,0.400000,8218624.0,7.829227e+04,0.000000,0.866667,0.066667,1.000000,0.952620
4,2013-06-30 22:23:24,1372631004,2,5851.9989,19.506663,0.333333,8218624.0,1.677720e+05,0.000000,0.200000,0.133333,1.000000,2.041364


Preparing Data to build the model

In [7]:
df_pr = df[['Time','CPU usage [%]']]

df_pr.columns = ['ds','y'] # To use prophet column names should be like that

train_data_pr = df_pr.iloc[:len(df)-5000]
test_data_pr = df_pr.iloc[len(df)-5000:]

Splitting data into train and test

In [8]:
train_data_pr.head()

,ds,y
0,2013-06-30 22:03:24,1.500000
1,2013-06-30 22:08:24,0.500000
2,2013-06-30 22:13:24,0.466667
3,2013-06-30 22:18:24,0.400000
4,2013-06-30 22:23:24,0.333333


In [9]:
test_data_pr.head()

,ds,y
20220,2013-09-12 11:50:05,0.500000
20221,2013-09-12 11:55:05,0.500000
20222,2013-09-12 12:00:05,2.166667
20223,2013-09-12 12:05:05,1.800000
20224,2013-09-12 12:10:06,1.900000


Building FB Prophet Model on the trained data

In [10]:
model = Prophet()
model.fit(train_data_pr)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [11]:
future = model.make_future_dataframe(periods=3,freq='MS')
prophet_pred = model.predict(future)
prophet_pred.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
20218,2013-09-12 11:40:05,1.404918,-1.886975,5.788772,1.404918,1.404918,0.545936,0.545936,0.545936,0.437310,0.437310,0.437310,0.108626,0.108626,0.108626,0.0,0.0,0.0,1.950853
20219,2013-09-12 11:45:05,1.405173,-2.048218,5.659355,1.405173,1.405173,0.565095,0.565095,0.565095,0.456984,0.456984,0.456984,0.108111,0.108111,0.108111,0.0,0.0,0.0,1.970268
20220,2013-10-01 11:45:05,2.802033,-35.484923,45.466365,-36.088105,44.249839,0.558629,0.558629,0.558629,0.456984,0.456984,0.456984,0.101645,0.101645,0.101645,0.0,0.0,0.0,3.360662
20221,2013-11-01 11:45:05,5.081120,-174.307721,201.198709,-177.246050,198.793171,0.377239,0.377239,0.377239,0.456984,0.456984,0.456984,-0.079744,-0.079744,-0.079744,0.0,0.0,0.0,5.458359
20222,2013-12-01 11:45:05,7.286688,-371.052261,412.348360,-370.799656,415.304097,0.387222,0.387222,0.387222,0.456984,0.456984,0.456984,-0.069762,-0.069762,-0.069762,0.0,0.0,0.0,7.673909


In [12]:
prophet_pred.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2013-06-30 22:03:24,0.343617,-3.616082,4.000173,0.343617,0.343617,-0.246042,-0.246042,-0.246042,-0.286538,-0.286538,-0.286538,0.040496,0.040496,0.040496,0.0,0.0,0.0,0.097575
1,2013-06-30 22:08:24,0.351670,-3.716545,3.973576,0.351670,0.351670,-0.255691,-0.255691,-0.255691,-0.296589,-0.296589,-0.296589,0.040898,0.040898,0.040898,0.0,0.0,0.0,0.095979
2,2013-06-30 22:13:24,0.359723,-3.746952,3.642785,0.359723,0.359723,-0.265732,-0.265732,-0.265732,-0.307025,-0.307025,-0.307025,0.041293,0.041293,0.041293,0.0,0.0,0.0,0.093991
3,2013-06-30 22:18:24,0.367776,-3.781577,4.064474,0.367776,0.367776,-0.276103,-0.276103,-0.276103,-0.317782,-0.317782,-0.317782,0.041679,0.041679,0.041679,0.0,0.0,0.0,0.091673
4,2013-06-30 22:23:24,0.375829,-3.649552,3.928828,0.375829,0.375829,-0.286739,-0.286739,-0.286739,-0.328796,-0.328796,-0.328796,0.042057,0.042057,0.042057,0.0,0.0,0.0,0.089091


In [13]:
len(prophet_pred)

20223

In [14]:
splitpred=prophet_pred.iloc[len(prophet_pred)-5000:]
len(splitpred)

5000

Evaluation Metrics

MAE

In [15]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(test_data_pr['y'], splitpred['yhat'])

1.674107389679477

RMSE

In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test_data_pr['y'], splitpred['yhat']))
rmse

2.406983960715526

Building Model for Forecast

In [17]:
m1 = Prophet(daily_seasonality=True)
m1.fit(train_data_pr)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [18]:
future1 = m1.make_future_dataframe(periods=2160, freq='H')
forecast1 = m1.predict(future1)
forecast1.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
22375,2013-12-11 07:45:05,8.009624,-467.785192,446.648747,-467.077585,446.473433,0.577240,0.577240,0.577240,0.293060,0.293060,0.293060,0.284180,0.284180,0.284180,0.0,0.0,0.0,8.586864
22376,2013-12-11 08:45:05,8.012687,-466.229669,445.511445,-467.390319,446.953439,0.730508,0.730508,0.730508,0.445822,0.445822,0.445822,0.284686,0.284686,0.284686,0.0,0.0,0.0,8.743195
22377,2013-12-11 09:45:05,8.015750,-466.131313,447.731856,-467.703054,447.433446,0.677429,0.677429,0.677429,0.393252,0.393252,0.393252,0.284177,0.284177,0.284177,0.0,0.0,0.0,8.693179
22378,2013-12-11 10:45:05,8.018814,-466.612036,447.369987,-468.015789,447.866206,0.613071,0.613071,0.613071,0.330386,0.330386,0.330386,0.282684,0.282684,0.282684,0.0,0.0,0.0,8.631884
22379,2013-12-11 11:45:05,8.021877,-466.703413,446.321509,-468.328523,448.093857,0.737231,0.737231,0.737231,0.456984,0.456984,0.456984,0.280247,0.280247,0.280247,0.0,0.0,0.0,8.759108


In [19]:
forecast1.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2013-06-30 22:03:24,0.343617,-3.653668,3.832623,0.343617,0.343617,-0.246042,-0.246042,-0.246042,-0.286538,-0.286538,-0.286538,0.040496,0.040496,0.040496,0.0,0.0,0.0,0.097575
1,2013-06-30 22:08:24,0.351670,-3.577833,3.592947,0.351670,0.351670,-0.255691,-0.255691,-0.255691,-0.296589,-0.296589,-0.296589,0.040898,0.040898,0.040898,0.0,0.0,0.0,0.095979
2,2013-06-30 22:13:24,0.359723,-3.845837,4.172421,0.359723,0.359723,-0.265732,-0.265732,-0.265732,-0.307025,-0.307025,-0.307025,0.041293,0.041293,0.041293,0.0,0.0,0.0,0.093991
3,2013-06-30 22:18:24,0.367776,-3.709572,4.130528,0.367776,0.367776,-0.276103,-0.276103,-0.276103,-0.317782,-0.317782,-0.317782,0.041679,0.041679,0.041679,0.0,0.0,0.0,0.091673
4,2013-06-30 22:23:24,0.375829,-3.554687,3.954030,0.375829,0.375829,-0.286739,-0.286739,-0.286739,-0.328796,-0.328796,-0.328796,0.042057,0.042057,0.042057,0.0,0.0,0.0,0.089091


In [20]:
len(forecast1)

22380

In [23]:
split=forecast1.iloc[len(forecast1)-2160:]
split.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
20220,2013-09-12 12:45:05,1.408236,-1.438368,6.494374,1.408236,1.408236,0.883188,0.883188,0.883188,0.781123,0.781123,0.781123,0.102065,0.102065,0.102065,0.0,0.0,0.0,2.291424
20221,2013-09-12 13:45:05,1.411299,-1.100629,6.587103,1.411299,1.411299,1.186715,1.186715,1.186715,1.090481,1.090481,1.090481,0.096234,0.096234,0.096234,0.0,0.0,0.0,2.598014
20222,2013-09-12 14:45:05,1.414363,-1.292404,6.345419,1.414363,1.414363,1.216646,1.216646,1.216646,1.126065,1.126065,1.126065,0.090581,0.090581,0.090581,0.0,0.0,0.0,2.631008
20223,2013-09-12 15:45:05,1.417426,-1.544379,6.389080,1.417426,1.417426,0.890235,0.890235,0.890235,0.805173,0.805173,0.805173,0.085062,0.085062,0.085062,0.0,0.0,0.0,2.307661
20224,2013-09-12 16:45:05,1.420489,-2.175267,5.731480,1.420489,1.420489,0.373435,0.373435,0.373435,0.293806,0.293806,0.293806,0.079629,0.079629,0.079629,0.0,0.0,0.0,1.793925


In [28]:
fbpredictedcpu = pd.DataFrame(split, columns=['ds'])
fbpredictedcpu = fbpredictedcpu.rename({'ds': 'Timestamp'}, axis=1)
fbpredictedcpu ['predicted_cpu_usage [%]' ] = split[['yhat']]
fbpredictedcpu


,Timestamp,predicted_cpu_usage [%]
20220,2013-09-12 12:45:05,2.291424
20221,2013-09-12 13:45:05,2.598014
20222,2013-09-12 14:45:05,2.631008
20223,2013-09-12 15:45:05,2.307661
20224,2013-09-12 16:45:05,1.793925
...,...,...
22375,2013-12-11 07:45:05,8.586864
22376,2013-12-11 08:45:05,8.743195
22377,2013-12-11 09:45:05,8.693179
22378,2013-12-11 10:45:05,8.631884


FB Prophet for Memory Usage

In [29]:
df_mem = df[['Time','Memory usage [%]']]

df_mem.columns = ['ds','y'] # To use prophet column names should be like that

train_data_mem = df_mem.iloc[:len(df)-5000]
test_data_mem = df_mem.iloc[len(df)-5000:]

In [30]:
train_data_mem.head()
len(train_data_pr)

20220

In [31]:
test_data_mem.head()
len(test_data_pr)

5000

In [32]:
modelmem = Prophet()
modelmem.fit(train_data_mem)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [33]:
future1 = modelmem.make_future_dataframe(periods=3,freq='MS')
prophet_predmem = modelmem.predict(future)
prophet_predmem.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
20218,2013-09-12 11:40:05,2.879534,-0.565163,7.729201,2.879534,2.879534,0.752420,0.752420,0.752420,0.606509,0.606509,0.606509,0.145910,0.145910,0.145910,0.0,0.0,0.0,3.631953
20219,2013-09-12 11:45:05,2.879900,-0.459677,7.772317,2.879900,2.879900,0.770242,0.770242,0.770242,0.623871,0.623871,0.623871,0.146371,0.146371,0.146371,0.0,0.0,0.0,3.650142
20220,2013-10-01 11:45:05,4.883537,-39.121075,48.189812,-39.731822,45.857871,0.965556,0.965556,0.965556,0.623871,0.623871,0.623871,0.341685,0.341685,0.341685,0.0,0.0,0.0,5.849093
20221,2013-11-01 11:45:05,8.152628,-188.427453,189.883666,-187.368030,189.720174,0.895209,0.895209,0.895209,0.623871,0.623871,0.623871,0.271339,0.271339,0.271339,0.0,0.0,0.0,9.047838
20222,2013-12-01 11:45:05,11.316266,-379.693512,378.892654,-381.558828,378.126767,-0.103855,-0.103855,-0.103855,0.623871,0.623871,0.623871,-0.727726,-0.727726,-0.727726,0.0,0.0,0.0,11.212410


In [34]:
prophet_predmem.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2013-06-30 22:03:24,1.413900,-2.892079,5.039002,1.413900,1.413900,-0.569830,-0.569830,-0.569830,-0.261492,-0.261492,-0.261492,-0.308339,-0.308339,-0.308339,0.0,0.0,0.0,0.844070
1,2013-06-30 22:08:24,1.421292,-3.215557,4.992104,1.421292,1.421292,-0.571661,-0.571661,-0.571661,-0.267332,-0.267332,-0.267332,-0.304328,-0.304328,-0.304328,0.0,0.0,0.0,0.849631
2,2013-06-30 22:13:24,1.428683,-3.462656,4.867722,1.428683,1.428683,-0.574047,-0.574047,-0.574047,-0.273732,-0.273732,-0.273732,-0.300315,-0.300315,-0.300315,0.0,0.0,0.0,0.854636
3,2013-06-30 22:18:24,1.436074,-3.371334,5.192480,1.436074,1.436074,-0.576950,-0.576950,-0.576950,-0.280651,-0.280651,-0.280651,-0.296299,-0.296299,-0.296299,0.0,0.0,0.0,0.859124
4,2013-06-30 22:23:24,1.443466,-3.328561,4.778167,1.443466,1.443466,-0.580330,-0.580330,-0.580330,-0.288050,-0.288050,-0.288050,-0.292280,-0.292280,-0.292280,0.0,0.0,0.0,0.863136


In [35]:
len(prophet_predmem)

20223

In [36]:
splitpredmem=prophet_predmem.iloc[len(prophet_predmem)-5000:]
len(splitpredmem)

5000

Evaluation Metrics

In [37]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(test_data_mem['y'], splitpredmem['yhat'])

2.977182828969469

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test_data_mem['y'], splitpredmem['yhat']))
rmse

3.98274327867045

Forecasting data for next three months with scale of one hour

In [ ]:
m2 = Prophet(daily_seasonality=True)
m2.fit(train_data_mem)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [ ]:
future2 = m2.make_future_dataframe(periods=2160, freq='H')
forecast2 = m2.predict(future2)
forecast2.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
22375,2013-12-11 07:45:05,12.353235,-426.714430,427.243940,-425.511563,431.787748,0.339231,0.339231,0.339231,0.198935,0.198935,0.198935,0.140296,0.140296,0.140296,0.0,0.0,0.0,12.692467
22376,2013-12-11 08:45:05,12.357629,-427.821330,430.563145,-425.860214,432.179879,0.579863,0.579863,0.579863,0.443665,0.443665,0.443665,0.136198,0.136198,0.136198,0.0,0.0,0.0,12.937492
22377,2013-12-11 09:45:05,12.362023,-426.977615,431.672666,-426.208865,432.572009,0.626776,0.626776,0.626776,0.494223,0.494223,0.494223,0.132553,0.132553,0.132553,0.0,0.0,0.0,12.988799
22378,2013-12-11 10:45:05,12.366417,-429.822795,431.383353,-426.557516,432.964139,0.626095,0.626095,0.626095,0.496773,0.496773,0.496773,0.129322,0.129322,0.129322,0.0,0.0,0.0,12.992513
22379,2013-12-11 11:45:05,12.370811,-426.155740,431.288614,-426.906168,433.356270,0.750341,0.750341,0.750341,0.623871,0.623871,0.623871,0.126470,0.126470,0.126470,0.0,0.0,0.0,13.121152


In [ ]:
forecast2.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2013-06-30 22:03:24,1.413900,-3.074556,4.610033,1.413900,1.413900,-0.569830,-0.569830,-0.569830,-0.261492,-0.261492,-0.261492,-0.308339,-0.308339,-0.308339,0.0,0.0,0.0,0.844070
1,2013-06-30 22:08:24,1.421292,-3.030827,4.843932,1.421292,1.421292,-0.571661,-0.571661,-0.571661,-0.267332,-0.267332,-0.267332,-0.304328,-0.304328,-0.304328,0.0,0.0,0.0,0.849631
2,2013-06-30 22:13:24,1.428683,-3.063271,4.737067,1.428683,1.428683,-0.574047,-0.574047,-0.574047,-0.273732,-0.273732,-0.273732,-0.300315,-0.300315,-0.300315,0.0,0.0,0.0,0.854636
3,2013-06-30 22:18:24,1.436074,-3.004575,4.709958,1.436074,1.436074,-0.576950,-0.576950,-0.576950,-0.280651,-0.280651,-0.280651,-0.296299,-0.296299,-0.296299,0.0,0.0,0.0,0.859124
4,2013-06-30 22:23:24,1.443466,-3.285968,4.816818,1.443466,1.443466,-0.580330,-0.580330,-0.580330,-0.288050,-0.288050,-0.288050,-0.292280,-0.292280,-0.292280,0.0,0.0,0.0,0.863136


In [ ]:
len(forecast2)

22380

In [ ]:
split1=forecast2.iloc[len(forecast2)-2160:]
len(split1) 

2160

In [ ]:
fbpredmem ['Timestamp'] = pd.DataFrame(split1, columns=['ds'])
fbpredmem.head()

,ds,Timestamp
20220,2013-09-12 12:45:05,2013-09-12 12:45:05
20221,2013-09-12 13:45:05,2013-09-12 13:45:05
20222,2013-09-12 14:45:05,2013-09-12 14:45:05
20223,2013-09-12 15:45:05,2013-09-12 15:45:05
20224,2013-09-12 16:45:05,2013-09-12 16:45:05


In [ ]:
del fbpredmem['ds']

In [ ]:
fbpredmem = fbpredictedcpu[['predicted_cpu_usage [%]']]
fbpredmem['predicted_memory_usage [%]'] = split1[['yhat']]

In [ ]:
fbpredmem['CPU capacity provisioned [MHZ]'] = 5851.9989
fbpredmem.head()

,predicted_cpu_usage [%],predicted_memory_usage [%],CPU capacity provisioned [MHZ]
20220,2.291424,3.936709,5851.9989
20221,2.598014,4.205725,5851.9989
20222,2.631008,4.231883,5851.9989
20223,2.307661,3.927900,5851.9989
20224,1.793925,3.427389,5851.9989


In [ ]:
fbpredmem['Memory capacity provisioned [KB]'] = 8218624.0
fbpredmem.head()

,predicted_cpu_usage [%],predicted_memory_usage [%],CPU capacity provisioned [MHZ],Timestamp,Memory capacity provisioned [KB]
20220,2.291424,3.936709,5851.9989,2013-09-12 12:45:05,8218624.0
20221,2.598014,4.205725,5851.9989,2013-09-12 13:45:05,8218624.0
20222,2.631008,4.231883,5851.9989,2013-09-12 14:45:05,8218624.0
20223,2.307661,3.927900,5851.9989,2013-09-12 15:45:05,8218624.0
20224,1.793925,3.427389,5851.9989,2013-09-12 16:45:05,8218624.0


In [ ]:
fbpredmem['CPU usage [MHZ]'] = (fbpredmem['predicted_cpu_usage [%]'].mul(fbpredmem['CPU capacity provisioned [MHZ]']/100))
fbpredmem.head()

,predicted_cpu_usage [%],predicted_memory_usage [%],CPU capacity provisioned [MHZ],Timestamp,Memory capacity provisioned [KB],CPU usage [MHZ]
20220,2.291424,3.936709,5851.9989,2013-09-12 12:45:05,8218624.0,134.094095
20221,2.598014,4.205725,5851.9989,2013-09-12 13:45:05,8218624.0,152.035771
20222,2.631008,4.231883,5851.9989,2013-09-12 14:45:05,8218624.0,153.966587
20223,2.307661,3.927900,5851.9989,2013-09-12 15:45:05,8218624.0,135.044317
20224,1.793925,3.427389,5851.9989,2013-09-12 16:45:05,8218624.0,104.980443


In [ ]:
fbpredmem['Memory usage [KB]']=(fbpredmem['predicted_memory_usage [%]'].mul(fbpredmem['Memory capacity provisioned [KB]']/100))

In [ ]:
fbpredmem['RMSE_CPU'] = 2.40698

In [ ]:
fbpredmem['MAE_CPU'] = 1.67410

In [ ]:
fbpredmem['RMSE_Memory'] = 3.98274

In [ ]:
fbpredmem['MAE_Memory'] = 2.97718

In [ ]:
fbpredmem['Model'] = 'FB Prophet'

In [ ]:
fbpredmem['VM'] = 1

In [ ]:
fbpredmem['CPU cores'] =2

In [ ]:
fbpredmem['Timestamp']=fbpredictedcpu[['Timestamp']]

In [ ]:
fbpredmem

,predicted_cpu_usage [%],predicted_memory_usage [%],CPU capacity provisioned [MHZ],Timestamp,Memory capacity provisioned [KB],CPU usage [MHZ],Memory usage [KB],RMSE_CPU,MAE_CPU,RMSE_Memory,MAE_Memory,Model,VM,CPU cores
20220,2.291424,3.936709,5851.9989,2013-09-12 12:45:05,8218624.0,134.094095,3.235433e+05,2.40698,1.6741,3.98274,2.97718,FB Prophet,1,2
20221,2.598014,4.205725,5851.9989,2013-09-12 13:45:05,8218624.0,152.035771,3.456527e+05,2.40698,1.6741,3.98274,2.97718,FB Prophet,1,2
20222,2.631008,4.231883,5851.9989,2013-09-12 14:45:05,8218624.0,153.966587,3.478026e+05,2.40698,1.6741,3.98274,2.97718,FB Prophet,1,2
20223,2.307661,3.927900,5851.9989,2013-09-12 15:45:05,8218624.0,135.044317,3.228193e+05,2.40698,1.6741,3.98274,2.97718,FB Prophet,1,2
20224,1.793925,3.427389,5851.9989,2013-09-12 16:45:05,8218624.0,104.980443,2.816842e+05,2.40698,1.6741,3.98274,2.97718,FB Prophet,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22375,8.586864,12.692467,5851.9989,2013-12-11 07:45:05,8218624.0,502.503158,1.043146e+06,2.40698,1.6741,3.98274,2.97718,FB Prophet,1,2
22376,8.743195,12.937492,5851.9989,2013-12-11 08:45:05,8218624.0,511.651668,1.063284e+06,2.40698,1.6741,3.98274,2.97718,FB Prophet,1,2
22377,8.693179,12.988799,5851.9989,2013-12-11 09:45:05,8218624.0,508.724750,1.067501e+06,2.40698,1.6741,3.98274,2.97718,FB Prophet,1,2
22378,8.631884,12.992513,5851.9989,2013-12-11 10:45:05,8218624.0,505.137763,1.067806e+06,2.40698,1.6741,3.98274,2.97718,FB Prophet,1,2


In [ ]:
fbpredmem.to_csv(r'/content/fb.csv', index = False)